In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('../Base_de_datos/Detalle_ventas.xlsx')
df

,id_venta,id_producto,nombre_producto,cantidad,precio_unitario,importe
0,1,90,Toallas Húmedas x50,1,2902,2902
1,2,82,Aceitunas Negras 200g,5,2394,11970
2,2,39,Helado Vainilla 1L,5,469,2345
3,2,70,Fernet 750ml,2,4061,8122
4,2,22,Medialunas de Manteca,1,2069,2069
...,...,...,...,...,...,...
338,118,70,Fernet 750ml,2,4061,8122
339,118,93,Cepillo de Dientes,3,2142,6426
340,118,50,Azúcar 1kg,2,727,1454
341,119,45,Fideos Spaghetti 500g,5,745,3725


# **1. Analizar problemas con el dataset**

### **Tipos de datos y valores nulos**

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_venta         343 non-null    int64 
 1   id_producto      343 non-null    int64 
 2   nombre_producto  343 non-null    object
 3   cantidad         343 non-null    int64 
 4   precio_unitario  343 non-null    int64 
 5   importe          343 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 16.2+ KB


### **Estadísticas generales**

In [4]:
df.describe(include='all')

,id_venta,id_producto,nombre_producto,cantidad,precio_unitario,importe
count,343.000000,343.000000,343,343.000000,343.000000,343.000000
unique,NaN,NaN,95,NaN,NaN,NaN
top,NaN,NaN,Queso Rallado 150g,NaN,NaN,NaN
freq,NaN,NaN,9,NaN,NaN,NaN
mean,61.492711,49.139942,NaN,2.962099,2654.495627,7730.078717
std,34.835525,29.135461,NaN,1.366375,1308.694720,5265.543077
min,1.000000,1.000000,NaN,1.000000,272.000000,272.000000
25%,31.000000,23.000000,NaN,2.000000,1618.500000,3489.000000
50%,61.000000,47.000000,NaN,3.000000,2512.000000,6702.000000
75%,93.000000,76.000000,NaN,4.000000,3876.000000,10231.500000


### **Conteo de valores faltantes**

In [5]:
df.isnull().sum()

id_venta           0
id_producto        0
nombre_producto    0
cantidad           0
precio_unitario    0
importe            0
dtype: int64

### **Análisis de duplicados (considerando relaciones)**

In [6]:
# Duplicados completos (todas las columnas iguales)
duplicados_completos = df.duplicated().sum()
print(f"Filas completamente duplicadas: {duplicados_completos}")

# Duplicados por combinación de producto en misma venta
duplicados_por_venta = df.duplicated(subset=['id_venta', 'id_producto']).sum()
print(f"Productos duplicados en la misma venta: {duplicados_por_venta}")

# Total de registros
print(f"Total de registros: {len(df)}")

Filas completamente duplicadas: 0
Productos duplicados en la misma venta: 4
Total de registros: 343


### **Verificación de integridad referencial**

In [7]:
# Verificar coherencia del importe
df['importe_calculado'] = df['cantidad'] * df['precio_unitario']
incoherencias = (df['importe'] != df['importe_calculado']).sum()
print(f"Registros con importe incoherente: {incoherencias}")

# Verificar valores negativos
cantidades_negativas = (df['cantidad'] <= 0).sum()
precios_negativos = (df['precio_unitario'] <= 0).sum()
print(f"Cantidades negativas o cero: {cantidades_negativas}")
print(f"Precios negativos o cero: {precios_negativos}")

Registros con importe incoherente: 0
Cantidades negativas o cero: 0
Precios negativos o cero: 0


# **2. Limpieza de datos**

### **Eliminar columna redundante**

In [8]:
df = df.drop(['nombre_producto'], axis=1)

### **Eliminación CUIDADOSA de duplicados**

In [9]:
# Solo eliminar duplicados COMPLETOS (todas las columnas iguales)
# NO eliminar por producto similar, ya que puede estar en ventas diferentes
df = df.drop_duplicates()

### **Eliminar filas con valores nulos críticos**

In [10]:
df = df.dropna(subset=['id_venta', 'id_producto', 'cantidad', 'precio_unitario'])

# **Estandarización**

### **Asegurar que todos los datos sigan el mismo formato:**
- IDs con formato numérico correcto
- Cantidades y precios con formato numérico
- Recalcular importe para consistencia

In [11]:
df['id_venta'] = pd.to_numeric(df['id_venta'], errors='coerce')
df['id_producto'] = pd.to_numeric(df['id_producto'], errors='coerce')
df['cantidad'] = pd.to_numeric(df['cantidad'], errors='coerce')
df['precio_unitario'] = pd.to_numeric(df['precio_unitario'], errors='coerce')
# Recalcular importe para consistencia
df['importe'] = df['cantidad'] * df['precio_unitario']

# **3. Normalización de datos**

### **Normalización de IDs y valores numéricos**

In [12]:
df['cantidad'] = df['cantidad'].abs()
df['precio_unitario'] = df['precio_unitario'].abs()
df['importe'] = df['importe'].abs()
# Eliminar columna auxiliar creada en análisis
df = df.drop(['importe_calculado'], axis=1, errors='ignore')

# **4. Validación Post-Limpieza y Normalización**

### **Verificar el resultado de la limpieza**

In [13]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   id_venta         343 non-null    int64
 1   id_producto      343 non-null    int64
 2   cantidad         343 non-null    int64
 3   precio_unitario  343 non-null    int64
 4   importe          343 non-null    int64
dtypes: int64(5)
memory usage: 13.5 KB


,id_venta,id_producto,cantidad,precio_unitario,importe
0,1,90,1,2902,2902
1,2,82,5,2394,11970
2,2,39,5,469,2345
3,2,70,2,4061,8122
4,2,22,1,2069,2069


# **5. Exportar datos limpios y normalizados**

In [17]:
df.to_csv('../Base_de_datos_limpia/Detalle_ventas_limpio.csv', index=False)